#1.Set-up

##1.1. Using Colab GPU for Learning

- First of all, we need to select GPU in the runtime menu

In [ ]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


- To check which GPU type you are using

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


##1.2. Install Hugging Face Library

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.3MB 4.6MB/s 
     |████████████████████████████████| 1.1MB 17.6MB/s 
     |████████████████████████████████| 2.9MB 41.7MB/s 
     |████████████████████████████████| 890kB 53.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=5e0e8c064238b027eaf8ac22e74a33bf7bb0e527462b955feb38102eb16672e6
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


#2.Corpus Data

##2.1.Mount Google Drive to this Notebook instance.

In [ ]:
# Mount Google Drive to this Notebook instance.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


- To check the location of this drive

In [ ]:
!ls

drive  sample_data


In [ ]:
!ls drive/My\ Drive/BERT/GH/TextQuality/Data

EssayTopic1.csv  EssayTopic2.csv


##2.2.Install required packages

In [ ]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

##2.3.Importing files to generate data

In [ ]:
topic = "1"
group = 7


fileDir = "drive/My Drive/BERT/GH/TextQuality/Data/EssayTopic"+topic+".csv"
fr = open(fileDir, 'r')
contents= fr.readlines()
fr.close()

train = pd.DataFrame(columns=('index', 'Label', 'Sentence'))
i = 0
index = ""
label = ""
sentence = ""
for content in contents:
    if i == 0:
        pass
    else:
        infos = content.split(",")
        index = infos[1]
        label = int(infos[2])
        sentence = infos[3].replace("\n","")
        if label == 0 or label == group:
          train.loc[i] = [index, label, sentence]
    i = i + 1

print(train)

    index Label                                           Sentence
1       0     0  인류의 개체수가 폭발적으로 증가하고 있는 상황에서 자연의 무조건적 보존은 쉬운 일도...
2       1     0  우선적으로 자연을 개발하는 것이 자연 또는 생태계를 확실하게 파괴하는 것이 아님을 ...
3       2     0   생태계를 지키며 개발할 수 있는 건설•토목적 기술들이 오늘날에는 충분히 제공되고 있다.
4       3     0  예를 들어 동물들의 이동 경로를 보장하는 고속도로 위의 생태 통로 보 근처의 어로 ...
5       4     0  자연 개발은 새로운 자원 창출에도 지대한 영향을 끼치는 요소가 될 수 있는데 중국에...
..    ...   ...                                                ...
176   175     7  왜냐하면 자연은 원래대로 보존하는 것이 자연에게 준종하기 뿐만 아니라 자연의 원모습...
177   176     7  물론 자연 개발도 좋지만 그 개발 후에 다른 문제점 생길지도 모르니까 좋지 않다고 ...
178   177     7      마찬 설형수술한 사람과 해 본 적이 없는 사람 비교하는 것은 똑같다고 생각합니다.
179   178     7  예를 들어 대부분 사람들은 “강남미인” 보다 “자연미인”은 더 선호하고 더 예쁜다고...
180   179     7                그러니까 자연 보존하는 것은 개발보다 더 중요한다고 생각합니다.

[120 rows x 3 columns]


##2.4.Cleaning the corpus data; remove the punctuations

In [ ]:
#정제하기

train['Sentence'] = train['Sentence'].str.replace(r'[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》\\n\t]+', " ", regex=True)
train['Sentence'] = train['Sentence'].str.replace(r'\t+', " ", regex=True)
train['Sentence'] = train['Sentence'].str.replace(r'[\\n]+'," ", regex=True)
train['Sentence'] = train['Sentence'].str.replace(r'  '," ", regex=True)
train['Sentence'] = train['Sentence'].str.replace(r'•'," ", regex=True)

In [ ]:
train.head(5)

,index,Label,Sentence
1,0,0,인류의 개체수가 폭발적으로 증가하고 있는 상황에서 자연의 무조건적 보존은 쉬운 일도...
2,1,0,우선적으로 자연을 개발하는 것이 자연 또는 생태계를 확실하게 파괴하는 것이 아님을 ...
3,2,0,생태계를 지키며 개발할 수 있는 건설 토목적 기술들이 오늘날에는 충분히 제공되고 있다
4,3,0,예를 들어 동물들의 이동 경로를 보장하는 고속도로 위의 생태 통로 보 근처의 어로 ...
5,4,0,자연 개발은 새로운 자원 창출에도 지대한 영향을 끼치는 요소가 될 수 있는데 중국에...


#3.Data processing


##3.1 Preprocessing

- Load the Tokenization.py for KoBERT

In [ ]:
import logging
import os
import unicodedata
from shutil import copyfile

from transformers import PreTrainedTokenizer


logger = logging.getLogger(__name__)

VOCAB_FILES_NAMES = {"vocab_file": "tokenizer_78b3253a26.model",
                     "vocab_txt": "vocab.txt"}

PRETRAINED_VOCAB_FILES_MAP = {
    "vocab_file": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/tokenizer_78b3253a26.model",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/tokenizer_78b3253a26.model",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/tokenizer_78b3253a26.model"
    },
    "vocab_txt": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/vocab.txt",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/vocab.txt",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/vocab.txt"
    }
}

PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES = {
    "monologg/kobert": 512,
    "monologg/kobert-lm": 512,
    "monologg/distilkobert": 512
}

PRETRAINED_INIT_CONFIGURATION = {
    "monologg/kobert": {"do_lower_case": False},
    "monologg/kobert-lm": {"do_lower_case": False},
    "monologg/distilkobert": {"do_lower_case": False}
}

SPIECE_UNDERLINE = u'▁'


class KoBertTokenizer(PreTrainedTokenizer):
    """
        SentencePiece based tokenizer. Peculiarities:
            - requires `SentencePiece <https://github.com/google/sentencepiece>`_
    """
    vocab_files_names = VOCAB_FILES_NAMES
    pretrained_vocab_files_map = PRETRAINED_VOCAB_FILES_MAP
    pretrained_init_configuration = PRETRAINED_INIT_CONFIGURATION
    max_model_input_sizes = PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES

    def __init__(
            self,
            vocab_file,
            vocab_txt,
            do_lower_case=False,
            remove_space=True,
            keep_accents=False,
            unk_token="[UNK]",
            sep_token="[SEP]",
            pad_token="[PAD]",
            cls_token="[CLS]",
            mask_token="[MASK]",
            **kwargs):
        super().__init__(
            unk_token=unk_token,
            sep_token=sep_token,
            pad_token=pad_token,
            cls_token=cls_token,
            mask_token=mask_token,
            **kwargs
        )

        # Build vocab
        self.token2idx = dict()
        self.idx2token = []
        with open(vocab_txt, 'r', encoding='utf-8') as f:
            for idx, token in enumerate(f):
                token = token.strip()
                self.token2idx[token] = idx
                self.idx2token.append(token)

        self.max_len_single_sentence = self.max_len - 2  # take into account special tokens
        self.max_len_sentences_pair = self.max_len - 3  # take into account special tokens

        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning("You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                           "pip install sentencepiece")

        self.do_lower_case = do_lower_case
        self.remove_space = remove_space
        self.keep_accents = keep_accents
        self.vocab_file = vocab_file
        self.vocab_txt = vocab_txt

        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(vocab_file)

    @property
    def vocab_size(self):
        return len(self.idx2token)

    def __getstate__(self):
        state = self.__dict__.copy()
        state["sp_model"] = None
        return state

    def __setstate__(self, d):
        self.__dict__ = d
        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning("You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                           "pip install sentencepiece")
        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(self.vocab_file)

    def preprocess_text(self, inputs):
        if self.remove_space:
            outputs = " ".join(inputs.strip().split())
        else:
            outputs = inputs
        outputs = outputs.replace("``", '"').replace("''", '"')

        if not self.keep_accents:
            outputs = unicodedata.normalize('NFKD', outputs)
            outputs = "".join([c for c in outputs if not unicodedata.combining(c)])
        if self.do_lower_case:
            outputs = outputs.lower()

        return outputs

    def _tokenize(self, text, return_unicode=True, sample=False):
        """ Tokenize a string. """
        text = self.preprocess_text(text)

        if not sample:
            pieces = self.sp_model.EncodeAsPieces(text)
        else:
            pieces = self.sp_model.SampleEncodeAsPieces(text, 64, 0.1)
        new_pieces = []
        for piece in pieces:
            if len(piece) > 1 and piece[-1] == str(",") and piece[-2].isdigit():
                cur_pieces = self.sp_model.EncodeAsPieces(piece[:-1].replace(SPIECE_UNDERLINE, ""))
                if piece[0] != SPIECE_UNDERLINE and cur_pieces[0][0] == SPIECE_UNDERLINE:
                    if len(cur_pieces[0]) == 1:
                        cur_pieces = cur_pieces[1:]
                    else:
                        cur_pieces[0] = cur_pieces[0][1:]
                cur_pieces.append(piece[-1])
                new_pieces.extend(cur_pieces)
            else:
                new_pieces.append(piece)

        return new_pieces

    def _convert_token_to_id(self, token):
        """ Converts a token (str/unicode) in an id using the vocab. """
        return self.token2idx.get(token, self.token2idx[self.unk_token])

    def _convert_id_to_token(self, index, return_unicode=True):
        """Converts an index (integer) in a token (string/unicode) using the vocab."""
        return self.idx2token[index]

    def convert_tokens_to_string(self, tokens):
        """Converts a sequence of tokens (strings for sub-words) in a single string."""
        out_string = "".join(tokens).replace(SPIECE_UNDERLINE, " ").strip()
        return out_string

    def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
        """
        Build model inputs from a sequence or a pair of sequence for sequence classification tasks
        by concatenating and adding special tokens.
        A RoBERTa sequence has the following format:
            single sequence: [CLS] X [SEP]
            pair of sequences: [CLS] A [SEP] B [SEP]
        """
        if token_ids_1 is None:
            return [self.cls_token_id] + token_ids_0 + [self.sep_token_id]
        cls = [self.cls_token_id]
        sep = [self.sep_token_id]
        return cls + token_ids_0 + sep + token_ids_1 + sep

    def get_special_tokens_mask(self, token_ids_0, token_ids_1=None, already_has_special_tokens=False):
        """
        Retrieves sequence ids from a token list that has no special tokens added. This method is called when adding
        special tokens using the tokenizer ``prepare_for_model`` or ``encode_plus`` methods.
        Args:
            token_ids_0: list of ids (must not contain special tokens)
            token_ids_1: Optional list of ids (must not contain special tokens), necessary when fetching sequence ids
                for sequence pairs
            already_has_special_tokens: (default False) Set to True if the token list is already formated with
                special tokens for the model
        Returns:
            A list of integers in the range [0, 1]: 0 for a special token, 1 for a sequence token.
        """

        if already_has_special_tokens:
            if token_ids_1 is not None:
                raise ValueError(
                    "You should not supply a second sequence if the provided sequence of "
                    "ids is already formated with special tokens for the model."
                )
            return list(map(lambda x: 1 if x in [self.sep_token_id, self.cls_token_id] else 0, token_ids_0))

        if token_ids_1 is not None:
            return [1] + ([0] * len(token_ids_0)) + [1] + ([0] * len(token_ids_1)) + [1]
        return [1] + ([0] * len(token_ids_0)) + [1]

    def create_token_type_ids_from_sequences(self, token_ids_0, token_ids_1=None):
        """
        Creates a mask from the two sequences passed to be used in a sequence-pair classification task.
        A BERT sequence pair mask has the following format:
        0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
        | first sequence    | second sequence
        if token_ids_1 is None, only returns the first portion of the mask (0's).
        """
        sep = [self.sep_token_id]
        cls = [self.cls_token_id]
        if token_ids_1 is None:
            return len(cls + token_ids_0 + sep) * [0]
        return len(cls + token_ids_0 + sep) * [0] + len(token_ids_1 + sep) * [1]

    def save_vocabulary(self, save_directory):
        """ Save the sentencepiece vocabulary (copy original file) and special tokens file
            to a directory.
        """
        if not os.path.isdir(save_directory):
            logger.error("Vocabulary path ({}) should be a directory".format(save_directory))
            return

        # 1. Save sentencepiece model
        out_vocab_model = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_file"])

        if os.path.abspath(self.vocab_file) != os.path.abspath(out_vocab_model):
            copyfile(self.vocab_file, out_vocab_model)

        # 2. Save vocab.txt
        index = 0
        out_vocab_txt = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_txt"])
        with open(out_vocab_txt, "w", encoding="utf-8") as writer:
            for token, token_index in sorted(self.token2idx.items(), key=lambda kv: kv[1]):
                if index != token_index:
                    logger.warning(
                        "Saving vocabulary to {}: vocabulary indices are not consecutive."
                        " Please check that the vocabulary is not corrupted!".format(out_vocab_txt)
                    )
                    index = token_index
                writer.write(token + "\n")
                index += 1

        return out_vocab_model, out_vocab_txt

- Convert each sentence to BERT format

In [ ]:
# 리뷰 문장 추출
sentences = train['Sentence']
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] 인류의 개체수가 폭발적으로 증가하고 있는 상황에서 자연의 무조건적 보존은 쉬운 일도 아닐 뿐더러 문명의 발전을 지연시키는 악재가 될 것이다  [SEP]',
 '[CLS] 우선적으로 자연을 개발하는 것이 자연 또는 생태계를 확실하게 파괴하는 것이 아님을 알아야 한다  [SEP]',
 '[CLS] 생태계를 지키며 개발할 수 있는 건설 토목적 기술들이 오늘날에는 충분히 제공되고 있다  [SEP]',
 '[CLS] 예를 들어 동물들의 이동 경로를 보장하는 고속도로 위의 생태 통로 보 근처의 어로 등은 많은 동물 개체수가 그것을 통해 이동하며 기본적인 기반 시설들과 생태계가 공존할 수 있음을 보여주고 있고 쓰레기 소각 매립장들도 환경을 보호할 수 있는 여러 가지 형태들로 개발되어 오히려 미적 가치를 추구하는 사회적 안프라로 탈바꿈하고 있다  [SEP]',
 '[CLS] 자연 개발은 새로운 자원 창출에도 지대한 영향을 끼치는 요소가 될 수 있는데 중국에서 대량으로 발견되고 있는 희토류나 아메리카 대륙의 셰일 가스 대양의 메탄 하이드레이트 등이 이를 확실하게 보여준다  [SEP]',
 '[CLS] 자원이 부족하다고 언급되는 현 시점에서의 대책 없는 개발 중단은 추가적인 자원 사용이 불가하므로 문명 발전의 지연과 원시화를 야기하게 되어 있다  [SEP]',
 '[CLS] 오히려 환경적인 개발로 인류의 발전과 자연 보존을 동시에 꾀하는 것이 옳을 것으로 생각된다  [SEP]',
 '[CLS] 자연보존이 더 중요하다고 생각합니다  [SEP]',
 '[CLS] 첫번째 이유는 자연을 보존하고 그 모습 그대로 생태계가 이뤄질 수 있도록 하는 것이 인간을 비롯한 모든 생태계의 안전을 보장하고 이익을 가져다주는 것이기 때문입니다  [SEP]',
 '[CLS] 두번째 이유는 자연을 보존하는 것이 생태계 종 다양성을 유지하는 일이기 때문입니다  [SEP]']

- Save the label data

In [ ]:
# 라벨 추출
labels = train['Label'].values
labels_re = []
for label in labels:
  labels_re.append(label)
labels = labels_re

- Check the KoBertTokenizer

In [ ]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1374: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
Setting 'max_len_single_sentence' is now deprecated. This value is automatically set up.
Setting 'max_len_sentences_pair' is now deprecated. This value is automatically set up.



[CLS] 인류의 개체수가 폭발적으로 증가하고 있는 상황에서 자연의 무조건적 보존은 쉬운 일도 아닐 뿐더러 문명의 발전을 지연시키는 악재가 될 것이다  [SEP]
['[CLS]', '▁인', '류', '의', '▁개', '체', '수가', '▁폭발', '적으로', '▁증가', '하고', '▁있는', '▁상황에서', '▁자연', '의', '▁무조건', '적', '▁보', '존', '은', '▁쉬', '운', '▁일', '도', '▁아', '닐', '▁뿐', '더', '러', '▁문', '명의', '▁발전', '을', '▁지연', '시키는', '▁악', '재', '가', '▁될', '▁것이다', '[SEP]']


In [ ]:
 # 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   2, 3758, 6107, 7095,  835, 7436, 6630, 4874, 7203, 4286, 7788,
       3860, 2691, 3916, 7095, 2117, 7202, 2355, 7264, 7086, 2923, 7010,
       3803, 5859, 3093, 5777, 2571, 5837, 6037, 2120, 6210, 2249, 7088,
       4330, 6725, 3133, 7191, 5330, 1772,  913,    3,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0])

In [ ]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])
print(labels)
print(input_ids)

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 7, 7, 7, 7, 7

In [ ]:

# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(input_ids)
train_labels = torch.tensor(labels)
train_masks = torch.tensor(attention_masks)	

print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])

tensor([   2, 3758, 6107, 7095,  835, 7436, 6630, 4874, 7203, 4286, 7788, 3860,
        2691, 3916, 7095, 2117, 7202, 2355, 7264, 7086, 2923, 7010, 3803, 5859,
        3093, 5777, 2571, 5837, 6037, 2120, 6210, 2249, 7088, 4330, 6725, 3133,
        7191, 5330, 1772,  913,    3,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0])
tensor(0)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1

In [ ]:
# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)


#4.Create model

##4.1. Load the KoBERT model; BertForSequenceClassification

In [ ]:
# 분류를 위한 BERT 모델 생성
model = BertForSequenceClassification.from_pretrained("monologg/kobert", num_labels=2)
model.cuda()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementw

##4.2. Create the functions for checking accuracy rate

In [ ]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

##4.3. Train the corpus data with KoBERT

In [ ]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 10

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 학습률을 조금씩 감소시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [ ]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

final_info = {}

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
      

    # 평가모드로 변경
    model.eval()
    train_input_ids = []
    train_input_mask = []
    train_labels = []

    num = 0
    for step, batch in enumerate(train_data):   #467, 128
      # print("batch",batch)
      # 배치를 GPU에 넣음
      batch = tuple(t.to(device) for t in batch)
      
      # 배치에서 데이터 추출
      b_input_ids, b_input_mask, b_labels = batch
      input_ids_arr = []
      input_mask_arr = []

      

      for i in range(0,len(b_input_ids)):
        input_ids_arr.append(int(b_input_ids[i]))
        input_mask_arr.append(int(b_input_mask[i]))

      
      train_input_ids.append(input_ids_arr)
      train_input_mask.append(input_mask_arr)
      train_labels.append(int(b_labels))


    train_input_ids = torch.tensor(train_input_ids)
    train_input_mask = torch.tensor(train_input_mask)
    train_labels = train_labels

    train_input_ids = train_input_ids.to(device)
    train_input_mask = train_input_mask.to(device)


    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(train_input_ids, 
                        token_type_ids=None, 
                        attention_mask=train_input_mask)
        

    sentence_vecs_sum = outputs[0]
    
    sentence_array = []
    for i in range(0,len(sentence_vecs_sum)):
      each_array = []
      for j in range(0,len(sentence_vecs_sum[i])):
        each_array.append(float(sentence_vecs_sum[i][j]))
      sentence_array.append(each_array)

    initial_df = pd.DataFrame(sentence_array)

    from sklearn.manifold import TSNE
    tsne = TSNE(n_components=2, random_state=0)
    tsne_obj= tsne.fit_transform(initial_df)

    tsne_df = pd.DataFrame({'X':tsne_obj[:,0],'Y':tsne_obj[:,0],'Label':train_labels})

    if epoch_i == 9:
      tsne_df.to_csv("drive/My Drive/BERT/GH/TextQuality/OutPut/t-SNE/Topic"+topic+"_tSNE_epoch_"+str(epoch_i)+"_"+str(group)+".csv")

    #t-sne
    #import numpy as np   
    #import pandas as pd 
    #from plotnine import *

    #print("")
    #print("  Network visualization  ")
    #print(ggplot(tsne_df, aes(x='X', y='Y')) + geom_point(aes(colour = 'Label')))

print("")
print("Training complete!")
print("")
print("Final result is below!")
print(final_info)



======== Epoch 1 / 10 ========
Training...


RuntimeError: ignored